This notebook is to explore and experiment on @radek1's notebook [simplified without need for chunking](https://www.kaggle.com/code/radek1/last-20-aids) and the [original code](https://www.kaggle.com/code/ttahara/last-aid-20) is by @ttahara

## rd: recsys - otto - last 20 aids - The purpose of this notebook - Grabbing the last 20 aid for each session of the test set, use them as prediction and submit or run local validation to see how powerful can the last 20 aids be.

## import dataset and functions

In [11]:
import os

try: import fastkaggle
except ModuleNotFoundError:
    os.system("pip install -Uq fastkaggle")

from fastkaggle import *

# use fastdebug.utils 
if iskaggle: os.system("pip install nbdev snoop")

if iskaggle:
    path = "../input/fastdebugutils0"
    import sys
    sys.path
    sys.path.insert(1, path)
    import utils as fu
    from utils import *
else: 
    from fastdebug.utils import *
    import fastdebug.utils as fu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.1 MB/s eta 0:00:00


ModuleNotFoundError: No module named 'utils'

In [1]:
import pandas as pd

train = pd.read_parquet('../input/otto-full-optimized-memory-footprint/train.parquet')
test = pd.read_parquet('../input/otto-full-optimized-memory-footprint/test.parquet')

!pip install pickle5
import pickle5 as pickle

with open('../input/otto-full-optimized-memory-footprint/id2type.pkl', "rb") as fh:
    id2type = pickle.load(fh)
with open('../input/otto-full-optimized-memory-footprint/type2id.pkl', "rb") as fh:
    type2id = pickle.load(fh)
    
sample_sub = pd.read_csv('../input/otto-recommender-system/sample_submission.csv')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.4/256.4 kB 5.0 MB/s eta 0:00:0000:01


In [2]:
sample_sub.head()

,session_type,labels
0,12899779_clicks,129004 126836 118524
1,12899779_carts,129004 126836 118524
2,12899779_orders,129004 126836 118524
3,12899780_clicks,129004 126836 118524
4,12899780_carts,129004 126836 118524


In [3]:
test.head()

,session,aid,ts,type
0,12899779,59625,1661724000,0
1,12899780,1142000,1661724000,0
2,12899780,582732,1661724058,0
3,12899780,973453,1661724109,0
4,12899780,736515,1661724136,0


### rd: recsys - otto - last 20 aid - sort df by two columns - test.sort_values(['session', 'ts'])

In [4]:
%%time

test = test.sort_values(['session', 'ts'])
test.head()

CPU times: user 3.16 s, sys: 704 ms, total: 3.86 s
Wall time: 3.87 s


,session,aid,ts,type
0,12899779,59625,1661724000,0
1,12899780,1142000,1661724000,0
2,12899780,582732,1661724058,0
3,12899780,973453,1661724109,0
4,12899780,736515,1661724136,0


### rd: recsys - otto - last 20 aid - take last 20 aids from each session - test.groupby('session')['aid'].apply(lambda x: list(x)[-20:])

In [5]:
session_aids = test.groupby('session')['aid'].apply(lambda x: list(x)[-20:])

In [12]:
session_aids.head()

session
12899779                                              [59625]
12899780           [1142000, 582732, 973453, 736515, 1142000]
12899781    [141736, 199008, 57315, 194067, 199008, 199008...
12899782    [889671, 1099390, 987399, 987399, 638410, 1072...
12899783    [255297, 1114789, 255297, 300127, 198385, 3001...
Name: aid, dtype: object

In [16]:
session_aids.index

Int64Index([12899779, 12899780, 12899781, 12899782, 12899783, 12899784,
            12899785, 12899786, 12899787, 12899788,
            ...
            14571572, 14571573, 14571574, 14571575, 14571576, 14571577,
            14571578, 14571579, 14571580, 14571581],
           dtype='int64', name='session', length=1671803)

### rd: recsys - otto - last 20 aid - loop through a Series with session as index with a list as the only column's value - for session, aids in session_aids.iteritems():

### rd: recsys - otto - last 20 aid - turn a list into a string with values connected with empty space - labels.append(' '.join([str(a) for a in aids]))

In [17]:
%%time

session_type = []
labels = []
session_types = ['clicks', 'carts', 'orders']

for session, aids in session_aids.iteritems():
    for st in session_types:
        session_type.append(f'{session}_{st}')
        labels.append(' '.join([str(a) for a in aids]))

CPU times: user 9.57 s, sys: 540 ms, total: 10.1 s
Wall time: 10.1 s


### rd: recsys - otto - last 20 aid - make a df from a dict with two lists as values - pd.DataFrame({'session_type': session_type, 'labels': labels})

In [18]:
submission = pd.DataFrame({'session_type': session_type, 'labels': labels})

In [19]:
submission.head()

,session_type,labels
0,12899779_clicks,59625
1,12899779_carts,59625
2,12899779_orders,59625
3,12899780_clicks,1142000 582732 973453 736515 1142000
4,12899780_carts,1142000 582732 973453 736515 1142000


In [ ]:
submission.to_csv('submission.csv', index=False)